In [1]:
from pathlib import Path
import matplotlib.pyplot as plt

from datagenerering import generate_ribs
plt.rcParams["font.family"] = "STIXGeneral"
plt.rcParams['mathtext.fontset'] = 'stix'
from matplotlib import animation, colors
import matplotlib as mpl
from matplotlib.patches import Rectangle

import numpy as np
# from scipy import interpolate
# from scipy.integrate import solve_ivp  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#r179348322575-1
from scipy.optimize import fsolve
import h5py
import re
import scipy.stats as stats

from hjelpefunksjonar import ranges, draw_rect
from rib import Rib
from constants import ρ

# from IPython.display import clear_output


from math import ceil, floor, hypot, log, sqrt

In [2]:
def get_reshape(dataset):
    """Ein metode som tek inn eit datasett og gjer alle reshapings-tinga for x og y, u og v pluss ribbelista.

    Args:
        dataset (string): filnamnet på hdf5-fila.

    Returns:
        tuple: Ein tuple med x, y, U, V (2d-arrays) og ribs (original-lista)
    """
 
    x,y,Umx,Vmx,I,J,ribs = get_xyuvijribs(dataset)
    
    x_reshape = x.reshape((J,I))
    y_reshape = y.reshape((J,I))
    
    Umx_reshape = Umx.reshape((len(Umx),J,I))
    Vmx_reshape = Vmx.reshape((len(Vmx),J,I))
    
    return x_reshape, y_reshape, Umx_reshape, Vmx_reshape, ribs

def get_xyuvijribs(dataset):
    """Ein metode som hentar ut alle dataa frå ei hdf5-fil og returnerer dei som ndarrays.

    Args:
        dataset (string): filnamnet på hdf5-fila

    Returns:
        tuple: x,y,U,V (alle som 1D-array), I,J (ints) og ribbelista
    """
    with h5py.File(dataset, 'r') as f:
        Umx = np.asarray(f['Umx'])
        Vmx = np.asarray(f['Vmx'])
        (I,J) = f.attrs['I'], f.attrs['J']
        x = np.asarray(f['x'])
        y = np.asarray(f['y'])
        ribs = np.asarray(f['ribs'])
    
    return x,y,Umx,Vmx,I,J,ribs
        
def get_mean(dataset):
    _,_,Umx,Vmx,I,J,_ = get_xyuvijribs(dataset)
    
    u_bar = np.nanmean(Umx,0).reshape((J,I))
    v_bar = np.nanmean(Vmx,0).reshape((J,I))
    
    
    return u_bar, v_bar
    

In [5]:
for case in Path("data/").glob("rib*[1-3].hdf5"):
    print(case)

data/rib25_Q60_2.hdf5
data/rib75_Q60_1.hdf5
data/rib75_Q100_1.hdf5
data/rib25_Q80_1.hdf5
data/rib25_Q20_2.hdf5
data/rib25_Q40_2.hdf5
data/rib50_Q60_1.hdf5
data/rib50_Q140_1.hdf5
data/rib25_Q100_2.hdf5
data/rib25_Q40_1.hdf5
data/rib25_Q60_1.hdf5
data/rib50_Q20_2.hdf5
data/rib75_Q80_1.hdf5
data/rib25_Q80_2.hdf5
data/rib50_Q100_1.hdf5
data/rib25_Q100_1.hdf5
data/rib50_Q120_1.hdf5
data/rib50_Q80_1.hdf5
data/rib75_Q100_2.hdf5
data/rib75_Q80_3.hdf5
data/rib75_Q20_1.hdf5
data/rib50_Q40_1.hdf5
data/rib75_Q80_2.hdf5
data/rib25_Q20_1.hdf5
data/rib50_Q20_1.hdf5
data/rib75_Q40_1.hdf5
data/rib50_Q20_3.hdf5
data/rib75_Q40_3.hdf5
data/rib25_Q20_3.hdf5
data/rib75_Q100_3.hdf5
data/rib75_Q40_2.hdf5


In [7]:
_,v_bar = get_mean("data/rib50_Q40_1.hdf5")

/tmp/ipykernel_383183/2094267812.py:43: RuntimeWarning: Mean of empty slice
  u_bar = np.nanmean(Umx,0).reshape((J,I))
/tmp/ipykernel_383183/2094267812.py:44: RuntimeWarning: Mean of empty slice
  v_bar = np.nanmean(Vmx,0).reshape((J,I))


In [8]:
v_bar.shape

(127, 126)

In [10]:
v_bar_utval = v_bar[:,38:107]